In [1]:
import pandas as pd
import requests
import xmltodict
import json
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from dataclasses import dataclass, asdict
import datamart

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
@dataclass
class NumberSearch:
    number: int = ''
    version: str = '2.1'
    enumeration_type: any = 'NPI-2'
def get_results(search):
    url = 'https://npiregistry.cms.hhs.gov/api/?version=2.1'
    parameters = asdict(search)
    response =  requests.get(url, params=parameters)
    results = response.json()
    return results
def get_npi_info(results, customer_number):
    result = results['results'][0]
    name = result['basic']['organization_name']
    npi = result['number']
    if result['other_names']:
        other_name = result['other_names'][0]['organization_name']
    else:
        other_name = "None"
    for address in result['addresses']:
        if address['address_purpose']=='LOCATION':
            location_address_1 = address['address_1']
            location_address_2 = address['address_2']
            location_zip = address['postal_code']
            location_state = address['state']
            location_city = address['city']
    new_dict = {
                'CUSTOMER_NUMBER':customer_number,
                'NAME':name,
                'OTHER_NAME': other_name,
                'NPI': npi,
                'LOCATION_ADDRESS_1': location_address_1,
                'LOCATION_ADDRESS_2': location_address_2,
                'LOCATION_ZIP': location_zip,
                'LOCATION_CITY': location_city,
                'LOCATION_STATE': location_state,
            }
    return new_dict

In [7]:
residents = pd.read_csv('../../Data/Credentialling/All_Residents.csv')

In [3]:
philly = pd.read_csv('../../Data/Credentialling/Philly_Place.csv')
philly = philly.fillna('None')

In [10]:
residents.columns

Index(['FULL_DT', 'ME', 'CUSTOMER_CATEGORY_DESC', 'CUSTOMER_NAME',
       'CUSTOMER_NBR', 'Company Name', 'Street 1ST', 'Street 2ND',
       'Street 3RD', 'City', 'State', 'Zip Code'],
      dtype='object')

In [11]:
pd.merge(philly, residents, left_on='Customer_Number', right_on='CUSTOMER_NBR')

,CUSTOMER_NAME_x,CUSTOMER_CATEGORY_DESC_x,Customer_Number,Street 1ST_x,Street 2ND_x,Street 3RD_x,City_x,State_x,Zip Code_x,Phone Number,...,CUSTOMER_CATEGORY_DESC_y,CUSTOMER_NAME_y,CUSTOMER_NBR,Company Name,Street 1ST_y,Street 2ND_y,Street 3RD_y,City_y,State_y,Zip Code_y
0,Holy Redeemer Hospital & Medical Center,Hospital,2188924,1648 Huntingdon Pike,None,None,Meadowbrook,PA,190468001,215 938-2066,...,Hospital,Holy Redeemer Hospital & Medical Center,2188924,Holy Redeemer Hospital & Medical Center,1648 Huntingdon Pike,NaN,NaN,Meadowbrook,PA,190468001.0
1,Holy Redeemer Hospital & Medical Center,Hospital,2188924,1648 Huntingdon Pike,None,None,Meadowbrook,PA,190468001,215 938-2066,...,Hospital,Holy Redeemer Hospital & Medical Center,2188924,Holy Redeemer Hospital & Medical Center,1648 Huntingdon Pike,NaN,NaN,Meadowbrook,PA,190468001.0
2,Holy Redeemer Hospital & Medical Center,Hospital,2188924,1648 Huntingdon Pike,None,None,Meadowbrook,PA,190468001,215 938-2066,...,Hospital,Holy Redeemer Hospital & Medical Center,2188924,Holy Redeemer Hospital & Medical Center,1648 Huntingdon Pike,NaN,NaN,Meadowbrook,PA,190468001.0
3,Holy Redeemer Hospital & Medical Center,Hospital,2188924,1648 Huntingdon Pike,None,None,Meadowbrook,PA,190468001,215 938-2066,...,Hospital,Holy Redeemer Hospital & Medical Center,2188924,Holy Redeemer Hospital & Medical Center,1648 Huntingdon Pike,NaN,NaN,Meadowbrook,PA,190468001.0
4,Holy Redeemer Hospital & Medical Center,Hospital,2188924,1648 Huntingdon Pike,None,None,Meadowbrook,PA,190468001,215 938-2066,...,Hospital,Holy Redeemer Hospital & Medical Center,2188924,Holy Redeemer Hospital & Medical Center,1648 Huntingdon Pike,NaN,NaN,Meadowbrook,PA,190468001.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,Bryn Mawr Rehab,Hospital,2087330,P O Box 3007,414 Paoli Pike,None,Malvern,PA,193553300,484 596-3935,...,Hospital,Bryn Mawr Rehab,2087330,Bryn Mawr Rehab,P O Box 3007,414 Paoli Pike,NaN,Malvern,PA,193553300.0
1212,Wills Surgery Center In Bucks County,Ambulatory Care,2100881,401 York Rd,None,None,Warminster,PA,189744508,215 443-3022,...,Ambulatory Care,Wills Surgery Center In Bucks County,2100881,Wills Surgery Center in Bucks County,401 York Rd,NaN,NaN,Warminster,PA,189744508.0
1213,Shriners Hospitals,Hospital,2086250,3551 N Broad St,None,None,Philadelphia,PA,191404131,215 430-4093,...,Hospital,Shriners Hospitals,2086250,Shriners Hospitals,3551 N Broad St,NaN,NaN,Philadelphia,PA,191404131.0
1214,New Britain Surgery Center,Ambulatory Care,2320965,1200 Manor Dr Ste 40,None,None,Chalfont,PA,189142203,267 954-1200,...,Ambulatory Care,New Britain Surgery Center,2320965,New Britain Surgery Center,1200 Manor Dr Ste 40,NaN,NaN,Chalfont,PA,189142203.0


In [10]:
dict_list =[]
for row in philly[0:50].itertuples():
    if row.NPI!='None':
        print(row.NPI)
        npi = NumberSearch(int(row.NPI))
        dict_list.append(get_npi_info(get_results(npi), row.Customer_Number))
pd.DataFrame(dict_list)

1750371522.0
1649401142.0
1215042114.0
1609417872.0
1669689931.0
1356333579.0
1629468137.0
1811992084.0
1386896306.0
1003850504.0
1780686931.0
1225091697.0
1275695355.0
1710355029.0
1821020421.0
1447214788.0
1215902168.0
1417946633.0
1912902206.0
1659875284.0
1356482970.0
1215916002.0
1972745941.0
1568459287.0
1508971441.0
1619176443.0
1205213824.0
1215116611.0
1780691386.0
1982880274.0
1811105190.0
1386746592.0
1932296696.0
1346669850.0
1477006849.0
1467440743.0
1932613643.0
1336378132.0
1073524518.0
1215921457.0
1942554308.0
1801059076.0
1992784821.0
1265664544.0
1790786218.0
1447282215.0
1386984052.0
1467510560.0
1275639023.0
1922238781.0


,CUSTOMER_NUMBER,NAME,OTHER_NAME,NPI,LOCATION_ADDRESS_1,LOCATION_ADDRESS_2,LOCATION_ZIP,LOCATION_CITY,LOCATION_STATE
0,2188924,HOLY REDEEMER HEALTH SYSTEM,HOLY REDEEMER HOSPITAL & MEDICAL CENTER,1750371522,1648 HUNTINGDON PIKE,,190468001,MEADOWBROOK,PA
1,2080722,UNIVERSITY OF PENNSYLVANIA HOSPITAL,None,1649401142,3400 SPRUCE ST,1-SILVERSTEIN,191044206,PHILADELPHIA,PA
2,2083485,"MAIN LINE HOSPITALS, INC.",PAOLI HOSPITAL,1215042114,255 W LANCASTER AVE,,193011763,PAOLI,PA
3,2085934,"METHODIST ASSOCIATES IN HEALTH CARE, INC",JEFFERSON COMMUNITY PHYSICIANS - RADIOLOGY,1609417872,4900 FRANKFORD AVE,,191242618,PHILADELPHIA,PA
4,2081089,ALBERT EINSTEIN HEALTHCARE NETWORK,ALBERT EINSTEIN MEDICAL CENTER,1669689931,5501 OLD YORK RD,,191413018,PHILADELPHIA,PA
5,2083375,CHESTER COUNTY HOSPITAL,None,1356333579,701 E MARSHALL ST,,193804412,WEST CHESTER,PA
6,2320977,"BELMONT BEHAVIORAL HOSPITAL, LLC",None,1629468137,4200 MONUMENT RD,,191311625,PHILADELPHIA,PA
7,2077717,ABINGTON MEMORIAL HOSPITAL,None,1811992084,1200 OLD YORK RD,,190013720,ABINGTON,PA
8,2085509,LANSDALE HOSPITAL CORPORATION,LANSDALE HOSPITAL,1386896306,100 MEDICAL CAMPUS DR,,194461259,LANSDALE,PA
9,2109124,"CHRISTIANA CARE HEALTH SERVICES, INC",CCHS CARDIAC SERVICES,1003850504,200 HYGEIA DR STE 1360,,197132049,NEWARK,DE


In [11]:
pd.DataFrame(dict_list).to_excel("../../Data/Credentialling/NPI_Sample.xlsx", index=False)

In [10]:
dict_list =[]
for row in philly.itertuples():
    if row.NPI!='None':
        print(row.NPI)
        npi = NumberSearch(int(row.NPI))
        dict_list.append(get_npi_info(get_results(npi), row.Customer_Number))

1750371522.0
1649401142.0
1215042114.0
1609417872.0
1669689931.0
1356333579.0
1629468137.0
1811992084.0
1386896306.0
1003850504.0
1780686931.0
1225091697.0
1275695355.0
1710355029.0
1821020421.0
1447214788.0
1215902168.0
1417946633.0
1912902206.0
1659875284.0
1356482970.0
1215916002.0
1972745941.0
1568459287.0
1508971441.0
1619176443.0
1205213824.0
1215116611.0
1780691386.0
1982880274.0
1811105190.0
1386746592.0
1932296696.0
1346669850.0
1477006849.0
1467440743.0
1932613643.0
1336378132.0
1073524518.0
1215921457.0
1942554308.0
1801059076.0
1992784821.0
1265664544.0
1790786218.0
1447282215.0
1386984052.0
1467510560.0
1275639023.0
1922238781.0
1881687614.0
1669616876.0
1295995785.0
1831139955.0
1659463669.0
1588603872.0
1265770572.0
1073770012.0
1952623936.0
1720182835.0
1649325713.0
1124261771.0
1730286097.0
1841263407.0
1538300579.0
1720262538.0
1740448711.0
1912561291.0
1295708832.0
1457324196.0
1558342329.0
1316174782.0
1366594699.0
1487857322.0
1548755838.0
1811932270.0
1679658710.0

In [15]:
pd.merge(philly, pd.DataFrame(dict_list), left_on='Customer_Number', right_on='CUSTOMER_NUMBER').to_csv('../../Data/Credentialling/Phily_Place_NPI.csv', index=False)

In [43]:
results = datamart.get_datamart_results((3,4,5,6,7,8),2020, customer_type = ['Hospital','Ambulatory Care'])

311563
141614
238807
16650


In [44]:
results = results[['FULL_DT', 'ME', 'ORDER_PRODUCT_ID','CUSTOMER_NBR', 'CUSTOMER_CATEGORY_DESC', 'PARTY_ID']]

In [19]:
philly_places = pd.merge(philly, pd.DataFrame(dict_list), left_on='Customer_Number', right_on='CUSTOMER_NUMBER')

In [45]:
results['CUSTOMER_NUMBER'] = [int(x)for x in results.CUSTOMER_NBR]

relevant = pd.merge(philly_places, results, on='CUSTOMER_NUMBER')

In [27]:
relevant

,CUSTOMER_NAME,CUSTOMER_CATEGORY_DESC_x,Customer_Number,Street 1ST,Street 2ND,Street 3RD,City,State,Zip Code,Phone Number,...,MAILING_STATE,LOCATION_ADDRESS,LOCATION_ZIP,LOCATION_CITY,LOCATION_STATE,FULL_DT,ME,ORDER_PRODUCT_ID,CUSTOMER_NBR,CUSTOMER_CATEGORY_DESC_y
0,Holy Redeemer Hospital & Medical Center,Hospital,2188924,1648 Huntingdon Pike,None,None,Meadowbrook,PA,190468001,215 938-2066,...,PA,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,2020-03-17,49598950040,4915513,000002188924,Hospital
1,Holy Redeemer Hospital & Medical Center,Hospital,2188924,1648 Huntingdon Pike,None,None,Meadowbrook,PA,190468001,215 938-2066,...,PA,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,2020-03-17,00802860331,4915513,000002188924,Hospital
2,Holy Redeemer Hospital & Medical Center,Hospital,2188924,1648 Huntingdon Pike,None,None,Meadowbrook,PA,190468001,215 938-2066,...,PA,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,2020-04-21,02405901377,4915513,000002188924,Hospital
3,Holy Redeemer Hospital & Medical Center,Hospital,2188924,1648 Huntingdon Pike,None,None,Meadowbrook,PA,190468001,215 938-2066,...,PA,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,2020-07-03,02507861973,4915513,000002188924,Hospital
4,Holy Redeemer Hospital & Medical Center,Hospital,2188924,1648 Huntingdon Pike,None,None,Meadowbrook,PA,190468001,215 938-2066,...,PA,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,2020-03-13,03305953414,4915513,000002188924,Hospital
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3620,Premier Surgical Center,Ambulatory Care,2086303,10000 Lincoln Dr E Ste 301,None,None,Marlton,NJ,80536801,856 797-8333,...,NJ,10000 LINCOLN DRIVE EAST,08053,MARLTON,NJ,2020-07-01,03375980460,4915514,000002086303,Ambulatory Care
3621,Best Impression Sugical Center,Ambulatory Care,2102208,1050 Dekalb Pike,None,None,Blue Bell,PA,194221812,610 213-3931,...,PA,1330 POWELL ST,194013353,NORRISTOWN,PA,2020-07-06,04101081597,4915514,000002102208,Ambulatory Care
3622,Best Impression Sugical Center,Ambulatory Care,2102208,1050 Dekalb Pike,None,None,Blue Bell,PA,194221812,610 213-3931,...,PA,1330 POWELL ST,194013353,NORRISTOWN,PA,2020-08-06,04115040249,4915513,000002102208,Ambulatory Care
3623,Camcare Health Corporation,Hospital,2099862,817 Federal St Ste 300,None,None,Camden,NJ,81031539,856 541-3270,...,NJ,1315 N DELAWARE ST,080661367,PAULSBORO,NJ,2020-08-21,01002931717,4915513,000002099862,Hospital


In [28]:
ppd = pd.read_csv("../../Data/PPD/ppd_data_20200718.csv")

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
ppd = ppd[['ME', 'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME']]

In [32]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
ppd['ME']=fix_me(list(ppd['ME']))

In [46]:
boop = pd.merge(relevant, ppd, on='ME')

In [35]:
product_key = pd.read_csv('../../Data/Credentialling/product_key.csv')

In [47]:
boop = pd.merge(product_key, boop, left_on='PRODUCT_ID', right_on='ORDER_PRODUCT_ID')

In [40]:
product_key.columns

Index(['PRODUCT_DESC', 'PRODUCT_ID'], dtype='object')

In [50]:
boop[['PRODUCT_DESC', 
       'CUSTOMER_CATEGORY_DESC_x', 'NPI_x', 'NAME',
       'OTHER_NAME', 'MAILING_ADDRESS', 'MAILING_ZIP', 'MAILING_CITY',
       'MAILING_STATE', 'LOCATION_ADDRESS', 'LOCATION_ZIP', 'LOCATION_CITY',
       'LOCATION_STATE', 'FULL_DT', 'ME', 'PARTY_ID', 'LAST_NAME', 'FIRST_NAME',
       'MIDDLE_NAME']].to_excel("../../Data/Credentialling/Philly_Ask.xlsx", index=False)